# REINFORCEMENT LEARNING: Problem set 2 - First problem computations
## Sergio-Yersi Villegas Pelegrín
### *June 2022*
#### **Dynamic Programming: in this notebook, we just run the loops with all the combinations of parameters to use in the first problem and export the results to csv files, in order to import them in another notebook which will contain the whole report.**

# Modules, functions and parameters

We will begin by importing the relevant modules that we will be using during this first problem set.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from numpy.random import choice
from datetime import datetime

Then, we define the generalized functions that we will be using throughout the exercises, which are briefly described below.

In [2]:
def action(a_type, q_low, q_high):
    
    """
    Given the type of service rate it gets
    as input, returns the corresponding q(a) value.
    
    a_type: type of service rate
    q_low: q value for low service rates
    q_high: q value for high service rates
    
    
    """
    if a_type == 'low':
        return q_low
    else:
        return q_high

def cost(a_type, q_low, q_high):
    """
    Given the type of service rate it gets
    as input, returns the corresponding cost value,
    associated to the action type. It calls the 
    above 'action' function.
    
    a_type: type of service rate
    q_low: q value for low service rates
    q_high: q value for high service rates
    
    """    
    a = action(a_type, q_low, q_high)
    if a == q_low:
        return 0
    else:
        return 0.01

def reward(a_type, q_low, q_high,x,N):
    """
    Given the type of service rate it gets
    as input, returns the corresponding reward value,
    associated to the corresponding action and cost
    which are computed by calling the previous functions,
    for each state x.
    
    a_type: type of service rate
    q_low: q value for low service rates
    q_high: q value for high service rates
    x: state
    N: maximum state (N-1 if starting at 0)
    
    """      
    c = cost(a_type, q_low, q_high)
    return -((x/N)**2)-c

def policy_type(N,pol_type):
    """
    Given the policy type,
    returns a vector with the 
    actions it will perform in 
    each state.
    
    N: maximum state (N-1 if starting at 0)
    pol_type: policy type
    
    """
    policy = np.full(N,'low', dtype='<U16')
    if pol_type == 'agg':
        for i in range(50,N):
            policy[i] = 'high'
    elif pol_type == 'high':
        policy = np.full(N,'high', dtype='<U16')
    return policy   

def feature_map(map_type, x, N):
    
    """
    Given the map type,
    returns a vector with the 
    corresponding feature map
    for the given state x.
    
    map_type: type of feature map
    x: state to compute the map
    N: maximum state (N-1 if starting at 0)
    
    """
        
    if map_type == 'fine':
        return [1 if i==x else 0 for i in range(N)]   
    elif map_type == 'coarse':
        return [1 if (x>=5*(i-1) and x<=5*i-1) else 0 for i in range(1,int(N/5)+1)]   
    else:
        phi1 = [1 if (x>=5*(i-1) and x<=5*i-1) else 0 for i in range(1,int(N/5)+1)]
        phi2 = [(x-5*(i-1))/5 if (x>=5*(i-1) and x<=5*i-1) else 0 for i in range(1,int(N/5)+1)]
        return phi1+phi2

def theta_func(map_type,x,N):
    """
    Given the map type,
    returns a vector of 0s with the 
    length of the corresponding
    feature map for the given state x.
    
    map_type: type of feature map
    x: state to compute the map
    N: maximum state (N-1 if starting at 0)
    
    """
    length = len(feature_map(map_type, x, N))
    return np.zeros((length))

Finally, we define the fixed parameters that are provided in the instructions.

In [3]:
N = 100 # Maximum amount of states
max_iter = 100 # Maximum number of iterations to perform
q_low = 0.51 # q value for low service rate
q_high = 0.6 # q value for high service rate
p = 0.5 # rate at which new requests arrive into the queue
gamma = 0.9 # Discount factor
s0 = 99 # Initial state (full queue)
iter_list = [10000,100000,1000000,10000000] # Sample transitions to try
map_list = ['fine','coarse','piecewise'] # Feature maps to try

# Problem 1

In this first problem, we will be computing the value functions associated to each policy type and comparing the obtained results for each one of them. We are going to be implementing the two methods we have been proposed: temporal difference and least squares temporal difference. Moreover, the two policies that will be analyzed are:

- Lazy policy: always using low service rate.
- Aggressive policy: using low or high service rate depending on wheter the queue is short (x<50) or long((x>=50), respectively.

Below, we describe the two methods inside each function and compute the results for all the possible combination of parameters: four different sample transitions values and three feature maps.

## Temporal difference

In [4]:
def temporal_difference(s0,N,pol_type,max_iter,a,b,q_low,q_high,map_type,p,gamma,optimal_policy=None):
    """
    Given a policy and an initial state, computes TD with the 'expected'
    next state and the corresponding feature maps. At each iteration, it updates
    the theta vector and re-defines the old values as the new ones, so the next
    iteration will use the right ones for both theta and the new state. When we have looped
    over all the iterations, we already have the final, updated theta, so we can finally
    create our approximated value function results for each state.
    
    s0: initial state
    N: maximum state (N-1 if starting at 0)
    pol_type: policy type
    max_iter: sample transitions
    a: tuning parameter
    b: tuning parameter
    q_low: q value for low service rates
    q_high: q value for high service rates
    map_type: type of feature map
    p: new requests arrival rate
    gamma: discount factor
    optimal policy: using an optimal policy if given (None as default)
    """
    policy = optimal_policy if optimal_policy is not None else policy_type(N, pol_type)
    theta = theta_func(map_type,s0,N)
    states = [i for i in range(N)]
    for k in range(1,max_iter):
        alpha = a/(k+b)
        q = action(policy[s0], q_low, q_high)
        if s0==N-1:
            p_dec = (1-p)*q
            p_eq = 1 - (1-p)*q
            s_prime = choice(a=[s0-1,s0], p=[p_dec,p_eq])
        elif s0==0:
            p_inc = p*(1-q)
            p_eq = 1-p*(1-q)
            s_prime = choice(a=[s0+1,s0], p=[p_inc,p_eq])
        else:
            p_dec = (1-p)*q
            p_inc = p*(1-q)
            p_eq = p*q+(1-p)*(1-q)
            s_prime = choice(a=[s0+1,s0-1,s0], p=[p_inc,p_dec,p_eq])
        f_x = feature_map(map_type, s0, N)
        f_next = feature_map(map_type, s_prime, N)
        delta = reward(policy[s0],q_low,q_high,s0,N) + gamma*np.dot(theta,f_next)-np.dot(theta,f_x)
        new_theta = theta + alpha*delta*np.array(f_x)
        theta = new_theta
        s0 = s_prime
    values = [np.dot(new_theta,feature_map(map_type, i, N)) for i in states]
    return values

## Least Squares Temporal Difference

In [5]:
def lstd(s0,N,pol_type,max_iter,a,b,q_low,q_high,map_type,p,gamma,optimal_policy=None):
    """
    Given a policy and an initial state, computes LSTD with the 'expected'
    next state and the corresponding feature maps. When we have looped
    over all the iterations, we compute the inverse matrix using the Moore–
    Penrose pseudoinverse and then the final, updated theta, so we can finally
    create our approximated value function results for each state.

    
    Uses same parameters as the function 'temporal_difference()'
    """
    policy = optimal_policy if optimal_policy is not None else policy_type(N, pol_type)
    theta = theta_func(map_type,s0,N)
    states = [i for i in range(N)]
    mat = 0
    b_T = 0
    for k in range(1,max_iter):
        alpha = a/(k+b)
        q = action(policy[s0], q_low, q_high)
        if s0==N-1:
            p_dec = (1-p)*q
            p_eq = 1 - (1-p)*q
            s_prime = choice(a=[s0-1,s0], p=[p_dec,p_eq])
        elif s0==0:
            p_inc = p*(1-q)
            p_eq = 1-p*(1-q)
            s_prime = choice(a=[s0+1,s0], p=[p_inc,p_eq])
        else:
            p_dec = (1-p)*q
            p_inc = p*(1-q)
            p_eq = p*q+(1-p)*(1-q)
            s_prime = choice(a=[s0+1,s0-1,s0], p=[p_inc,p_dec,p_eq])
        f_x = np.matrix(feature_map(map_type, s0, N))
        f_next = np.matrix(feature_map(map_type, s_prime, N))
        mat += f_x.T*((f_x-gamma*f_next))
        b_T += reward(policy[s0],q_low,q_high,s0,N)*f_x
        s0 = s_prime
    mat=np.linalg.pinv(mat)
    theta = b_T*mat
    values = [np.dot(theta,feature_map(map_type, i, N)) for i in states]
    return values

## Results

In [8]:
def results_td(s0,N,pol_type_agg,pol_type_lazy,max_iter,a,b,q_low,q_high,map_type,p,gamma,optimal_policy=None):
    """
    Loops over all the combinations of the parameters
    and stores the results in a list, which will be 
    later transformed into a csv file.
    
    Uses same parameters as the function 'temporal_difference()'
    """
    results_list = []
    for i_iter in max_iter:
        for i_map in map_type:
            now = datetime.now()
            lazy_values = temporal_difference(s0,N,pol_type_agg,i_iter,a,b,q_low,q_high,\
                                              i_map,p,gamma,optimal_policy=None)
            agg_values = temporal_difference(s0,N,pol_type_lazy,i_iter,a,b,q_low,q_high,\
                                              i_map,p,gamma,optimal_policy=None)
            y = np.array(lazy_values) - np.array(agg_values)
            end = datetime.now()
            print((end-now).total_seconds())            
            results_list.append(y)
    return results_list

def results_lstd(s0,N,pol_type_agg,pol_type_lazy,max_iter,a,b,q_low,q_high,map_type,p,gamma,optimal_policy=None):
    results_list = []
    for i_iter in max_iter:
        for i_map in map_type:
            now = datetime.now()
            lazy_values = lstd(s0,N,pol_type_agg,i_iter,a,b,q_low,q_high,\
                                              i_map,p,gamma,optimal_policy=None)
            agg_values = lstd(s0,N,pol_type_lazy,i_iter,a,b,q_low,q_high,\
                                              i_map,p,gamma,optimal_policy=None)
            y = np.array(lazy_values) - np.array(agg_values)
            end = datetime.now()
            print((end-now).total_seconds())
            results_list.append(y)
    return results_list

In [9]:
td = results_td(s0=s0,N=N,pol_type_agg='agg',pol_type_lazy='lazy',max_iter=iter_list,a=100000,b=100000,\
                                  q_low=q_low, q_high=q_high, map_type = map_list,p=p, gamma=gamma)
td = np.matrix(td).T
td = pd.DataFrame(td,columns=['$10^4$fine','$10^4$coarse','$10^4$piece','$10^5$fine','$10^5$coarse','$10^5$piece',\
                           '$10^6$fine','$10^6$coarse','$10^6$piece','$10^7$fine','$10^7$coarse','$10^7$piece'])

1.28346
0.616169
0.865699
12.386387
5.78161
8.246673
123.277626
55.875429
82.339507
1228.914347
559.841303
824.966363


In [10]:
td.to_csv('/Volumes/GoogleDrive/Mi unidad/BSE/TERM 3/RL/data/td_results.csv', index=False)

In [11]:
lstd = results_lstd(s0=s0,N=N,pol_type_agg='agg',pol_type_lazy='lazy',max_iter=iter_list,a=100000,b=100000,\
                                  q_low=q_low, q_high=q_high, map_type = map_list,p=p, gamma=gamma)
lstd = np.matrix(lstd).T
lstd = pd.DataFrame(lstd,columns=['$10^4$fine','$10^4$coarse','$10^4$piece','$10^5$fine','$10^5$coarse','$10^5$piece',\
                           '$10^6$fine','$10^6$coarse','$10^6$piece','$10^7$fine','$10^7$coarse','$10^7$piece'])



1.683942
1.010543
1.148188
16.142467
9.107121
11.237954
161.617774
91.253084
112.090488
1617.195363
917.542994
1122.793047


ValueError: matrix must be 2-dimensional

The error above appeared in the second equality used, but does not affect the loop performed. In the next cells, we fix the error and correctly store the results. We have not re-run the previous cell again since it takes a lot of time and the error does not affect the final result.

In [35]:
correct_lstd = []
for i in lstd:
    correct_lstd.append(i.flatten())

In [36]:
correct_lstd = np.matrix(correct_lstd).T
correct_lstd = pd.DataFrame(correct_lstd,columns=['$10^4$fine','$10^4$coarse','$10^4$piece','$10^5$fine','$10^5$coarse','$10^5$piece',\
                           '$10^6$fine','$10^6$coarse','$10^6$piece','$10^7$fine','$10^7$coarse','$10^7$piece'])

correct_lstd



,$10^4$fine,$10^4$coarse,$10^4$piece,$10^5$fine,$10^5$coarse,$10^5$piece,$10^6$fine,$10^6$coarse,$10^6$piece,$10^7$fine,$10^7$coarse,$10^7$piece
0,-0.003113,-0.018044,0.000486,0.000012,0.001088,0.000009,0.000031,-0.000002,0.000006,0.000014,0.000260,-0.000006
1,-0.004442,-0.018044,0.000852,0.000061,0.001088,0.000078,0.000026,-0.000002,0.000036,0.000025,0.000260,-0.000011
2,-0.007482,-0.018044,0.001218,-0.000069,0.001088,0.000148,-0.000004,-0.000002,0.000066,0.000036,0.000260,-0.000016
3,-0.011691,-0.018044,0.001584,0.000158,0.001088,0.000218,-0.000149,-0.000002,0.000096,0.000066,0.000260,-0.000021
4,-0.017688,-0.018044,0.001950,0.000562,0.001088,0.000288,-0.000194,-0.000002,0.000126,0.000061,0.000260,-0.000026
...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.088985,0.396875,-0.106850,0.318339,2.060911,-0.343890,0.171111,0.632153,-0.263856,1.796739,1.644064,-0.106105
96,0.027141,0.396875,0.494481,0.526807,2.060911,-0.080201,0.496688,0.632153,-0.021223,2.829010,1.644064,0.424393
97,0.219062,0.396875,1.095811,0.935751,2.060911,0.183489,1.008821,0.632153,0.221411,3.328125,1.644064,0.954891
98,0.533383,0.396875,1.697142,1.283941,2.060911,0.447178,2.006022,0.632153,0.464044,4.248231,1.644064,1.485389


In [37]:
correct_lstd.to_csv('/Volumes/GoogleDrive/Mi unidad/BSE/TERM 3/RL/data/lstd_results.csv', index=False)